In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


c:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [5]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [6]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ"]
raceDB = build_dataset('gru_inputs_new2_pir_simple_NO_SP.npy', hidden_size ,state_filter=states, margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()
#raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

['speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_']
(1192161, 21)
(126312, 24)
Latest date = 2023-03-29 00:00:00
num_features_per_dog=14


  0%|          | 0/3551 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/16728 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


number of races = 16728, number of unique dogs = 3551
0        (Palmerston North, 410.0)
1        (Palmerston North, 410.0)
2        (Palmerston North, 410.0)
3        (Palmerston North, 410.0)
4        (Palmerston North, 410.0)
                   ...            
16723    (Palmerston North, 375.0)
16724    (Palmerston North, 457.0)
16725    (Palmerston North, 375.0)
16726    (Palmerston North, 457.0)
16727    (Palmerston North, 410.0)
Length: 16728, dtype: object


In [7]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 13565, Test examples 3163


In [8]:
start_date = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").date()
end_date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()-datetime.timedelta(1)
period = start_date
batches = []
while start_date<end_date:
   
    print(f"{start_date=}")
    period = min(start_date+datetime.timedelta(365), end_date)
    print(f"{period=}")
    batches.append((start_date,period))
    start_date=period
    #batches.append((start_date,end_date))
# batches.append((period,end_date))
print(batches)

start_date=datetime.date(2019, 12, 1)
period=datetime.date(2020, 11, 30)
start_date=datetime.date(2020, 11, 30)
period=datetime.date(2021, 11, 30)
start_date=datetime.date(2021, 11, 30)
period=datetime.date(2022, 7, 31)
[(datetime.date(2019, 12, 1), datetime.date(2020, 11, 30)), (datetime.date(2020, 11, 30), datetime.date(2021, 11, 30)), (datetime.date(2021, 11, 30), datetime.date(2022, 7, 31))]


In [9]:
batch_races_ids = [] # list of race_ids
j = 0
current_batch = []
for i,r in enumerate(raceDB.raceIDs):   
    _,end_date = batches[j]
    race_date = raceDB.racesDict[r].race_date
    if race_date>end_date:
        print(end_date)

        batch_races_ids.append(current_batch)
        current_batch = []
        j += 1
        if j>len(batches)-1:
            break
    else:
        current_batch.append(r)
print(f"Train examples {[len(x) for x in batch_races_ids]}")

2020-11-30
2021-11-30
2022-07-31
Train examples [5026, 5218, 3258]


In [10]:
train_dogs = []
train_dog_input = []
for bi, batch in enumerate(tqdm(batch_races_ids)):
    print(bi)
    print(batch)
    batch_dogs = []
    batch_dog_input = []
    for i in raceDB.dog_ids:
        dog = raceDB.dogsDict[i]
        train = [dog.races[x] for x in dog.races.keys() if x in batch]
        if train:
            batch_dogs.append(dog)
            batch_dog_input.append(train)
    train_dogs.append(batch_dogs)
    train_dog_input.append(batch_dog_input)

  0%|          | 0/3 [00:00<?, ?it/s]

0
['500539129', '500539130', '500539131', '500539132', '500539133', '500539134', '500539135', '500539136', '500539137', '500539138', '500539139', '500539140', '500539141', '500539142', '500539143', '500539144', '500541168', '500541169', '500541170', '500541171', '500541172', '500541173', '500541174', '500541175', '500541176', '500541177', '500541178', '500541179', '500541180', '500541181', '500541182', '500543140', '500543141', '500543142', '500543143', '500543144', '500543145', '500543146', '500543147', '500543148', '500543149', '500543150', '500543151', '500543152', '501809115', '501809116', '501809117', '501809118', '501809119', '501809120', '501809121', '501809122', '501809123', '501809124', '501809125', '501809126', '501809127', '501809128', '501809129', '502271152', '502271153', '502271154', '502271155', '502271156', '502271157', '502271158', '502271159', '502271160', '502271161', '502271162', '502271163', '502271164', '502271165', '502271166', '502271167', '502271168', '50256366

In [11]:
batch_races = [[raceDB.racesDict[r] for r in inner] for inner in batch_races_ids]

In [12]:
print(f"Train examples {[len(x) for x in train_dogs]}")
print(f"Train examples {[len(x) for x in train_dog_input]}")
print(f"Train examples {[len(x) for x in batch_races]}")
print(f"Train examples {[len(x) for x in batch_races_ids]}")

Train examples [1943, 1885, 1544]
Train examples [1943, 1885, 1544]
Train examples [5026, 5218, 3258]
Train examples [5026, 5218, 3258]


In [13]:
raceDB.create_dogs_test_split_date()

  0%|          | 0/3551 [00:00<?, ?it/s]

In [14]:
raceDB.attach_races_to_dog_inputv2()

  0%|          | 0/3551 [00:00<?, ?it/s]

In [15]:
test_idx = range(0,len(raceDB.test_dog_ids))
packed_x = [pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_y = pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)

In [16]:
# train_dog_input_stacked = [[torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in x]] for x in train_dog_input]

In [17]:
# [sum([len(x) for x in n])/len([len(x) for x in n]) for n in train_dog_input_stacked]

In [18]:
raceDB.batches = {'num_batches':len(train_dogs),'dogs':train_dogs,'train_dog_input':train_dog_input,'batch_races':batch_races,'batch_races_ids':batch_races_ids,'packed_x':packed_x, 'packed_y':packed_y}

In [27]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = rnn_classes.GRUNetv2(84,config['hidden_size'], num_layers=config['num_layers'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file

      raceDB.to_cuda()

      # criterion = nn.SmoothL1Loss(reduction='mean', beta=5)
      criterion = nn.CrossEntropyLoss(reduction='none')
      # optimizer = optim.AdamW(model.parameters(), lr=0.0001, set_t)
      # optimizer = optim.Adadelta(model.parameters())
      optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])

      model = model.to(device)

      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt, Exception) as error:
        print(error)
        print("finished Early")
        
      dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict_gru_v6 , model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [20]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'stats':raceDB.stats_cols, 'num_layers':2,'batch_size': 750, 'dropout': 0.3, 'epochs': 10_000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0005, 'loss': 'L1', 'l1_beta':0.1,   'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':7}

In [26]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [22]:
raceDB.reset_hidden()

In [23]:
torch.cuda.empty_cache()

In [28]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='drawn-snowflake-346')

{'hidden_size': 32, 'stats': ['speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_'], 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0005, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
10000
{'hidden_size': 32, 'stats': ['speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_'], 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0005, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
GRUNetv2(
  (gru): GRU(84, 32, num_layers=2)
  (batchnorm): BatchNorm1d(84, eps=1e-05, mom

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Exec time of t1 -2.135324999995646
Exec time of t2 1.1049319000012474
Exec time of hidden 0.005498099999385886
Exec time of hidden2 2.9380977999971947
Exec time of hidden2 0.0363538000092376
Exec time of t_p2 0.01199429998814594
Exec time of t_p3 0.0029145000007702038
Exec time of t_p3 6.093540700006997
Exec time of t_p3 6.09481070000038
Exec time of t_p3 6.095803700009128
Exec time of t1 -2.010163200000534
Exec time of t2 1.0431071000057273
Exec time of hidden 0.005304200007230975
Exec time of hidden2 3.0360888999857707
Exec time of hidden2 0.038874200006830506
Exec time of t_p2 0.009083800003281794
Exec time of t_p3 0.0033862999989651144
Exec time of t_p3 6.4214606000023196
Exec time of t_p3 6.422377800001414
Exec time of t_p3 6.4232442999928026
Exec time of t1 -1.475513500001398
Exec time of t2 0.8722680000064429
Exec time of hidden 0.00455659998988267
Exec time of hidden2 1.8734128000069177
Exec time of hidden2 0.02240469999378547
Exec time of t_p2 0.008752400011871941
Exec time of

  0%|          | 0/3 [00:00<?, ?it/s]

...

Exec time of t1 -2.1449789999896893
Exec time of t2 1.1578518000023905
Exec time of hidden 0.007368500009761192
Exec time of hidden2 3.104059099990991
Exec time of hidden2 0.03825599999981932
Exec time of t_p2 0.009109300008276477
Exec time of t_p3 0.003300700002000667
Exec time of t_p3 10.96323670000129
Exec time of t_p3 10.964337400000659
Exec time of t_p3 10.965437599996221
Exec time of t1 -1.8141930000128923
Exec time of t2 1.0997768999950495
Exec time of hidden 0.005982599992421456
Exec time of hidden2 3.1147302000026684
Exec time of hidden2 0.03631850000238046
Exec time of t_p2 0.00879259999783244
Exec time of t_p3 0.0032344000064767897
Exec time of t_p3 9.707373400000506
Exec time of t_p3 9.708311800000956
Exec time of t_p3 9.709130000002915
Exec time of t1 -1.8828089000016917
Exec time of t2 0.9996880999970017
Exec time of hidden 0.004326700000092387
Exec time of hidden2 2.2976794999995036
Exec time of hidden2 0.03038770001148805
Exec time of t_p2 0.01041429999168031
Exec time 

  0%|          | 0/3 [00:00<?, ?it/s]

Exec time of t1 -1.9757561000005808
Exec time of t2 1.1878036999987671
Exec time of hidden 0.008338300001923926
Exec time of hidden2 3.242836699995678
Exec time of hidden2 0.03824769999482669
Exec time of t_p2 0.009090200008358806
Exec time of t_p3 0.003116099993349053
Exec time of t_p3 7.144570399992517
Exec time of t_p3 7.145496999990428
Exec time of t_p3 7.1464787000004435
Exec time of t1 -2.0729785000003176
Exec time of t2 1.03614209999796
Exec time of hidden 0.005882400000700727
Exec time of hidden2 2.997036000000662
Exec time of hidden2 0.03633069999341387
Exec time of t_p2 0.008912200006307103
Exec time of t_p3 0.0035887999983970076
Exec time of t_p3 6.695040899998276
Exec time of t_p3 6.695998499999405
Exec time of t_p3 6.696884599994519
Exec time of t1 -1.6324769999919226
Exec time of t2 0.8497313000116264
Exec time of hidden 0.007161199988331646
Exec time of hidden2 1.8823025000019697
Exec time of hidden2 0.02469310000014957
Exec time of t_p2 0.008926299997256137
Exec time of

  0%|          | 0/3 [00:00<?, ?it/s]

Exec time of t1 -2.1222097000136273
Exec time of t2 1.2084396999998717
Exec time of hidden 0.00910079998720903
Exec time of hidden2 3.7824701000063214
Exec time of hidden2 0.04310460000124294
Exec time of t_p2 0.009648399995057844
Exec time of t_p3 0.003320600008009933


In [34]:
race = raceDB.get_race_input([1,2])[0]

In [35]:
race

In [37]:
outputs = [race.race_dist]+[race.race_track]+[d.hidden_out for d in race.dogs]
race.hidden_in = torch.cat(outputs)

In [38]:
race.hidden_in

tensor([ 4.1000e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0

In [ ]:
race.hidden_in.shape

In [ ]:
torch.rand([1395, 3, 64])

In [ ]:
x = torch.rand([1395, 3, 64])
x[100]

In [ ]:
x.transpose(0,1)

In [ ]:
torch.zeros(3,5,10)

In [ ]:
len_test = len(raceDB.test_dog_ids)
test_idx = range(0,len_test)

# dog_inputs = [[z.full_input for z in inner] for inner in [x for x in raceDB.train_dogs.values()]]
# Uses train dogs here because only dogs in Train set will have new hidden values?? Does that make sense?
# no will jumble around but wont throw error in feeding model, because when building dogs sorted, 
# lengths will be made to match size of X(test)
# Shouldn't have much of an effect due to size of test set??
dogs = [x for x in  raceDB.test_dogs.values()]  #[Dog]
dog_input = [inner for inner in [x for x in raceDB.get_dog_test(test_idx)]] #[[DogInput]]

# for dog in dogs:
#     dog.hidden = dog.hidden.to('cuda:0')

train = [torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]]
X = pack_sequence(train, enforce_sorted=False).to('cuda:0')

dogs_sorted = [dogs[x] for x in X[3]]

In [ ]:
hidden_in = torch.stack([x.hidden for x in dogs])

In [ ]:
hidden_in.shape

In [ ]:
hidden_in.reshape((3, 1439, 64))

In [ ]:
dogs = [x for x in  raceDB.test_dogs.values()][:5]

In [ ]:
[d.hidden for d in dogs]

In [ ]:
set(dog.races.keys()).intersection(set(raceDB.test_race_ids))
set(dog.races.keys()).intersection(set(raceDB.test_race_ids))

In [ ]:
[r.race.raceid for r in dog.test]

In [ ]:
for i in set(dog.races.keys()).intersection(set(raceDB.test_race_ids)):
    print(i)

In [ ]:
[dog.races[x] for x in set(dog.races.keys()).intersection(set(raceDB.test_race_ids))]

In [ ]:
dog.test

In [ ]:
dog_input = [inner for inner in [x for x in raceDB.get_dog_train([0,1,3,4,5,6])]][2]

In [ ]:
races = [r.race for r in dog_input]

In [ ]:
pd.Series([r.race_date for r in races])

In [ ]:
raceDB.create_hidden_states_dict_v2()

In [ ]:
model_saver(model,optimizer,450,0.2,raceDB.hidden_states_dict_gru_v6, model_name="long run to use")

In [ ]:
raceDB.hidden_states_dict_gru_v6

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='none')

In [ ]:
training_testing_gru.validate_model_v2(model,raceDB,criterion)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
len_test = len(raceDB.test_dog_ids)
test_idx = range(0,len_test)

# dog_inputs = [[z.full_input for z in inner] for inner in [x for x in raceDB.train_dogs.values()]]
dogs = [x for x in  raceDB.test_dogs.values()]  #[Dog]
dog_input = [inner for inner in [x for x in raceDB.get_dog_test(test_idx)]] #[[DogInput]]

for dog in dogs:
    dog.hidden = dog.hidden.to('cuda:0')

train = [torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]]
X = pack_sequence(train, enforce_sorted=False)

dogs_sorted = [dogs[x] for x in X[3]]
hidden_in = torch.unsqueeze(torch.stack([x.hidden for x in dogs_sorted]).to('cuda:0'),0)

In [ ]:
batchnorm = torch.nn.BatchNorm1d(196)

In [ ]:
batchnorm(X)

In [ ]:
loss = nn.CrossEntropyLoss(reduction='none')
X = torch.rand(5,8)
Y = torch.rand(5,8)
loss(X,Y)

In [ ]:
for i,dog in enumerate(dog_input):
    dog_outputs = output[i]
    for j,di in enumerate(dog):
        di.output = dog_outputs[j]

In [ ]:
raceDB.margin_from_dog_to_race()

In [ ]:
torch.tensor(100.1).to('cuda:0')

In [ ]:
torch.tensor([100.1]).to('cuda:0')

In [ ]:
for r in raceDB.test_race_ids:
    race = raceDB.racesDict[r]
    outputs = []
    for d in race.dogs:
        print(d)
        print(d.output)
        outputs.append(d.output)
    break
    print(output)
    race.margins = torch.stack(outputs)

In [ ]:
torch.cat(outputs)

In [ ]:
output[0]

In [ ]:
test_races = [raceDB.racesDict[x] for x in raceDB.test_race_ids][0]

In [ ]:
test

In [ ]:
print(test_races.margins)

In [ ]:
[x.dog.dog_name for x in test_races.dogs]

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":64}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [100]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [50,125,250,500]
        },
        "batch_before_backwards": {
            'values': [5,10,15,50]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

In [ ]:

stop
model = GRUNet(203,64).to(device)
model.load_state_dict(model_state_dict)

In [ ]:

dataset.create_hidden_states_dict()